# Wikipedia similarity graph

The goal of this notebook is to create a graph where node are wikipedia page and weighted edges are the similarity between articles.


In [104]:
import numpy as np
import wikipedia_preprocessing as wiki
from sklearn.metrics.pairwise import cosine_similarity
import pickle
from os import path
import re
from nltk.tokenize import word_tokenize
from matplotlib import pyplot as plt

DATA_folder = path.join("Data", "Wikipedia")

## Computing similarity

### Step 1: Fetching wikipedia pages data

In [66]:
wiki.get_all_dumps()
wiki.precompute_pages(max_dumps=1)

File  exists: aborting download (use force=True to download anyway).
Pages already precomputed: aborting (use force=True).


#### Filtering categories

In [67]:
reg_category = re.compile("^\s*Category:.*\s*$")
reg_redirect = re.compile("REDIRECT")

def get_categories(page):
    categories = []
    for line in page["content"].split("\n"):
        if reg_redirect.match(line):
            return []
        if reg_category.match(line):
            categories.append(line.strip())
    return categories


In [68]:
def all_categories(pages):
    categories = {}
    i = 0
    for p in pages:
        print("\r%s articles computed" % i, end="")
        i += 1
        page_cat = get_categories(p)
        for cat in page_cat:
            if cat in categories:
                categories[cat].append(p["name"])
            else:
                categories[cat] = [p["name"]]
    remove = [cat for cat, l in categories.items() if len(l) < 2]
    for cat in remove:
        del categories[cat]
    print("\nRemoved %d categories with 1 article" % len(remove))
    return categories

#### Filtering articles

In [69]:
pages = wiki.Pages()
cats = all_categories(pages)
goodcat = [cat for cat, l in cats.items() if len(l) > 50]

def get_connected_articles(cats, goodcats):
    articles = set()
    for cat in goodcats:
        for art in cats[cat]:
            articles.add(art)
    return list(articles)

def reorder_names(names):
    pages = wiki.Pages()
    new_names = []
    for p in pages:
        if p["name"] in names:
            new_names.append(p["name"])
    return new_names

    
names=reorder_names(get_connected_articles(cats, goodcat))
len(names)
Ncorp = len(names)
print("%d articles kept" % Ncorp)

19821 articles computed
Removed 43798 categories with 1 article
2480 articles kept


#### Creating reference matrix

In [75]:
def create_cat_matrix(goodcats, names):
    Ncorp = len(names)
    cat_matrix = np.zeros((Ncorp, Ncorp))
    for cat in goodcats:
        for i in range(len(cats[cat])):
            for j in range(1, i):
                x = names.index(cats[cat][i])
                y = names.index(cats[cat][j])
                cat_matrix[x, y] = cat_matrix[y, x] = 1
    return cat_matrix

cat_matrix = create_cat_matrix(goodcat, names)

#### Create reference graph

In [ ]:
graph_filepath = path.join(DATA_folder, "precomp", "categories.dot")
def save_dot_file(names, cats, goodcats, filename):
    with open(filename, "w") as f:
        f.write("graph graphname {")
        f.write("node [shape=point, fontsize=0]")
        for c in goodcats:
            for i, a in enumerate(cats[c]):
                for j in range(i + 1, len(cats[c])):
                    x = names.index(cats[c][i])
                    y = names.index(cats[c][j])
                    if x < 200 and y < 200:
                        f.write("a%d -- a%d;" % (x, y))
        f.write("}")

save_dot_file(names, cats, goodcat, graph_filepath)

In [78]:
np.sum(cat_matrix[:100, :100])

1022.0

### Step 2: Compute Word2vec of each page

In [79]:
Nvec = 300
word2vec = wiki.get_word2vec()

Loading Word2vec model.
Loaded from pickle


In [81]:
filepath = path.join(DATA_folder, "precomp", "pages_vectorize_cats.vec.gz")
pages = wiki.Pages()
wiki.vectorize_pages_filtered(pages, word2vec, names, filepath)

Calculating vectorized pages
Computed 2479 pages

/home/mulcyber/Projets/wikipediaNLP/wikipedia_preprocessing.py:648: RuntimeWarning: divide by zero encountered in true_divide
  idf = np.log(Ncorp / appears)


### Step 3: List vocabulary

In [71]:
voc = word2vec.index2entity

### Step 4: Compute inverse document frequency (idf) of each term

Done during preprocessing of doc_vector

### Step 5: For each document, compute each term frequency (tf) and tf-idf. Keep only the N=100 most important terms.

In [94]:
N = 100
pages = wiki.Pages()
idf_filepath = path.join(DATA_folder, "precomp", "idf.pkl")

def compute_tf(d, idf):
    tokens = word_tokenize(d["content"])
    n = len(tokens)
    f = {}
    terms = []
    idf_t = []
    for t in tokens:
        if word2vec.vocab.get(t):
            if t not in terms:
                terms.append(t)
                idf_t.append(idf[word2vec.vocab.get(t).index])
                f[t] = 1
                if idf_t[-1] == np.inf:
                    print("IDF inf for %s" % t)
            else:
                f[t] += 1         
    terms = np.array(terms)
    f = np.array([f[t] for t in terms])
    f = 0.5 + 0.5 * f / np.max(f)
    idf_t = np.array(idf_t)
    return terms, f * idf_t

def compute_doc_vectors():
    D = wiki.Pages()
    with open(idf_filepath, "rb") as f:
        idf = pickle.load(f)
    doc_vectors = np.zeros((Ncorp, N, Nvec + 1))
    i = 0
    for d in D:
        if d["name"] in names:
            print("\r%d/%d" % (i, Ncorp), end="")
            terms, tfidf = compute_tf(d, idf)
            index_sorted = (-tfidf).argsort()
            terms = terms[index_sorted]
            tfidf = tfidf[index_sorted]
            for j in range(min(N, len(terms))):
                doc_vectors[i, j, :-1] = word2vec.get_vector(terms[j])
            doc_vectors[i, :min(N, len(terms)), -1] = tfidf[:N]
            i += 1
    return doc_vectors

vec_doc_filepath = path.join(DATA_folder, "vec_doc.pkl")

if path.isfile(vec_doc_filepath):
    with open(vec_doc_filepath, "rb") as f:
        doc_vectors = pickle.load(f)
    print("Loaded doc vectors")
else:
    doc_vectors = compute_doc_vectors()
    with open(vec_doc_filepath, "wb") as f:
        pickle.dump(doc_vectors, f)


2479/2480

### Testing: creating a search function using this similarity

In [95]:
with open(idf_filepath, "rb") as f:
        idf = pickle.load(f)

In [96]:
def search_similar(name, sim_matrix, names, N=10):
    matches = np.argsort(-sim_matrix[names.index(name), :])
    return [names[matches[i]] for i in range(N)]

print(search_similar('Albert Einstein', sim_matrix, names, N=10))
print(search_similar('Apollo 11', sim_matrix, names, N=10))

/usr/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """Entry point for launching an IPython kernel.


[('hearthstone', 1.0),
 ('fire-place', 0.6301341652870178),
 ('easy-chair', 0.6231460571289062),
 ('hearth', 0.613952100276947),
 ('tea-table', 0.6105866432189941),
 ('writing-table', 0.6100817322731018),
 ('court-yard', 0.5942029356956482),
 ('farm-house', 0.59410160779953),
 ('bed-room', 0.5908367037773132),
 ('livingroom', 0.5757289528846741)]

In [ ]:
def get_words(name, names, doc_vectors, N=10):
    return [word2vec.similar_by_vector(doc_vectors[names.index(name), i, :Nvec])[0] for i in range(N)]
    
get_words('Albert Einstein', names, doc_vectors, N=10)

## Creating the graph

### Step1: Compute all document similarities

In [122]:
def doc_similarity1(d1, d2):
    sim = 0
    for v1 in d1[:N]:
        t1 = v1[:Nvec]
        for v2 in d2[:N]:
            t2 = v2[:Nvec]
            sim += cosine_similarity([t1], [t2]) / N**2
    return sim


def doc_similarity2(d1, d2):
    sim = 0
    for v1 in d1[:N]:
        t1 = v1[:Nvec]
        max_sim = 0
        for v2 in d2[:N]:
            t2 = v2[:Nvec]
            curr_sim = cosine_similarity([t1], [t2])
            if curr_sim > max_sim:
                max_sim = curr_sim
        sim += max_sim / N
    return sim


def doc_similarity3(d1, d2):
    sim = 0
    for v1 in d1[:N]:
        t1 = v1[:Nvec]
        tfidf1 = v1[-1]
        for v2 in d2[:N]:
            t2 = v2[:Nvec]
            tfidf2 = v2[-1]
            sim += cosine_similarity([t1], [t2]) * tfidf1 * tfidf2 / N**2
    return sim


def doc_similarity4(d1, d2):
    sim = 0
    for v1 in d1[:N]:
        t1 = v1[:Nvec]
        tfidf1 = v1[-1]
        max_sim = 0
        for v2 in d2[:N]:
            t2 = v2[:Nvec]
            tfidf2 = v2[-1]
            curr_sim = cosine_similarity([t1], [t2]) * tfidf1 * tfidf2
            if curr_sim > max_sim:
                max_sim = curr_sim
            sim += max_sim / N
    return sim


def similarity_matrix(doc_vec, sim_func):
    Ncorp = len(doc_vec)
    sim_matrix = np.zeros((Ncorp, Ncorp))
    for i in range(Ncorp):
        print("\r%s articles computed" % i, end="")
        for j in range(i):
            sim_matrix[i, j] = sim_matrix[j, i] = sim_func(doc_vec[i], doc_vec[j])
        if i and i % 10 == 0:
            with open(sim_matrix_path % i, "wb") as f:
                pickle.dump(sim_matrix[:i, :i], f)
    return sim_matrix

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim1_small_matrix_%d.pkl")
N = 100
Nvec = 300
sim_matrix = similarity_matrix(doc_vectors[:201], doc_similarity1)

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim2_matrix_%d.pkl")
sim_matrix = similarity_matrix(doc_vectors[:101], doc_similarity2)

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim3_matrix_%d.pkl")
sim_matrix = similarity_matrix(doc_vectors[:101], doc_similarity3)

55 articles computed

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim4_matrix_%d.pkl")
sim_matrix = similarity_matrix(doc_vectors[:101], doc_similarity4)

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim1_small_matrix_%d.pkl")
N = 20
Nvec = 20
sim_matrix = similarity_matrix(doc_vectors[:201], doc_similarity1)

### Step2: Compute similarity histogram

In [ ]:
sim_matrix_path = path.join(DATA_folder, "precomp", "sim1_matrix_%d.pkl")

with open(sim_matrix_path % 100, "rb") as f:
    sim_matrix = pickle.load(f)

In [ ]:
plt.figure(figsize=(8, 5))
plt.hist(sim_matrix[sim_matrix != 0].reshape((-1, 1)), 40)
plt.xlabel("Similarity")
plt.ylabel("N")
plt.savefig(path.join(DATA_folder, "img", "hist1_%d.png" % len(sim_matrix)))
plt.show()

### Step 3: Choose threshold

In [123]:
def true_pos_rate(ref_matrix, sim_matrix, threshold):
    return np.sum(ref_matrix[sim_matrix > threshold]) / np.sum(ref_matrix)

def false_pos_rate(ref_matrix, sim_matrix, threshold):
    return np.sum(1 - ref_matrix[sim_matrix > threshold]) / np.sum(1 - ref_matrix)

def RoC_curve(ref_matrix, sim_matrix, thresholds):
    true_pos_rates = np.zeros(len(thresholds))
    false_pos_rates = np.zeros(len(thresholds))
    for i, t in enumerate(thresholds):
        true_pos_rates[i] = true_pos_rate(ref_matrix, sim_matrix, t)
        false_pos_rates[i] = false_pos_rate(ref_matrix, sim_matrix, t)
    return true_pos_rates, false_pos_rates


def plot_curve(ref_matrix, sim_matrix, a, b, n=20, filename=None):
    thresholds = np.arange(a, b, (b - a)/n)
    true_pos_rates, false_pos_rates = RoC_curve(ref_matrix, sim_matrix, thresholds)
    f1 = 2 * true_pos_rates * (1 - false_pos_rates) / (true_pos_rates + 1 - false_pos_rates)
    fig, ax1 = plt.subplots(figsize=(8, 5))
    ax2 = ax1.twinx()
    ax1.plot(false_pos_rates, true_pos_rates)
    ax2.plot(false_pos_rates, f1, "r")
    ax1.plot([0, 1], [0, 1], "b--")
    ax2.plot(false_pos_rates[np.argmax(f1)], np.max(f1), "ro")
    ax2.annotate("t = %.2f" % thresholds[np.argmax(f1)], (false_pos_rates[np.argmax(f1)], np.max(f1) + 0.03))
    ax1.set_xlabel("FPR: False positive rate")
    ax1.set_ylabel("TPR: True positive rate")
    ax2.set_ylabel("F1 score")
    ax1.set_xlim([0, 1])
    ax1.set_ylim([0, 1])
    ax2.set_ylim([0, 1])
    if filename:
        plt.savefig(filename)
    else:
        plt.show()


In [ ]:
sim_matrix[sim_matrix == 0] = np.min(sim_matrix[sim_matrix != 0])
Nmat = len(sim_matrix)
ref_matrix = cat_matrix[:Nmat, :Nmat]
a, b = -0.10, 0.15
plot_curve(ref_matrix, sim_matrix, a, b, n=100, filename=path.join(DATA_folder, "img", "ROC1_small_%d.png" % len(sim_matrix)))

### Step 4: Create graph

In [109]:
sim_graph_filepath = path.join(DATA_folder, "precomp", "similarity200.dot")
def sim_save_dot_file(matrix, threshold, filename):
    with open(filename, "w") as f:
        N = len(matrix)
        f.write("graph graphname {")
        f.write("node [shape=point, fontsize=0]")
        for i in range(N):
            for j in range(i + 1, N):
                if matrix[i, j] > threshold:
                    f.write("a%d -- a%d;" % (i, j))
        f.write("}")

sim_save_dot_file(sim_matrix, 42, sim_graph_filepath)

## Results

### Graph representation

Done using the sdpf command of graphviz

### Comparision to wikipedia portals

See RoC curves